In [1]:
val cubed = (s:Long)=>{
    s*s*s
}

Intitializing Scala interpreter ...

Spark Web UI available at http://L2108017.bosonit.local:4041
SparkContext available as 'sc' (version = 3.0.3, master = local[*], app id = local-1634294462128)
SparkSession available as 'spark'


cubed: Long => Long = $Lambda$1892/0x0000000801416420@162e6b2f


In [2]:
spark.udf.register("cubed",cubed)

res0: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$1892/0x0000000801416420@162e6b2f,LongType,List(Some(class[value[0]: bigint])),Some(cubed),false,true)


In [3]:
spark.range(1,9).createOrReplaceTempView("udf_test")

In [4]:
spark.sql("SELECT id, cubed(id) AS id_cubed FROM udf_test").show()

+---+--------+
| id|id_cubed|
+---+--------+
|  1|       1|
|  2|       8|
|  3|      27|
|  4|      64|
|  5|     125|
|  6|     216|
|  7|     343|
|  8|     512|
+---+--------+



In [5]:
val t1 = Array(35, 36, 32, 30, 40, 42, 38)
val t2 = Array(31, 32, 34, 55, 56)
val tC = Seq(t1, t2).toDF("celsius")
tC.createOrReplaceTempView("tC")

t1: Array[Int] = Array(35, 36, 32, 30, 40, 42, 38)
t2: Array[Int] = Array(31, 32, 34, 55, 56)
tC: org.apache.spark.sql.DataFrame = [celsius: array<int>]


In [6]:
tC.show(false)

+----------------------------+
|celsius                     |
+----------------------------+
|[35, 36, 32, 30, 40, 42, 38]|
|[31, 32, 34, 55, 56]        |
+----------------------------+



In [8]:
// La función reduce no existe en Jupyter Notebook

/*
spark.sql("""
SELECT celsius, 
 reduce(
 celsius, 
 0, 
 (t, acc) -> t + acc, 
 acc -> (acc div size(celsius) * 9 div 5) + 32
 ) as avgFahrenheit 
 FROM tC
""").show(false)
*/

//Este sería el equivalente usando transform
spark.sql("""
SELECT celsius,
       transform (
           celsius,
           t -> ((t * 9) div 5) + 32
           ) as fahrenheit_temp       
FROM tC
""").show(10,false)

+----------------------------+-------------------------------+
|celsius                     |fahrenheit_temp                |
+----------------------------+-------------------------------+
|[35, 36, 32, 30, 40, 42, 38]|[95, 96, 89, 86, 104, 107, 100]|
|[31, 32, 34, 55, 56]        |[87, 89, 93, 131, 132]         |
+----------------------------+-------------------------------+



In [19]:
import org.apache.spark.sql.functions._
val delaysPath = "C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
val airportsPath = "C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/airport-codes-na.txt"

import org.apache.spark.sql.functions._
delaysPath: String = C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv
airportsPath: String = C:/Users/alvaro.romero/Big_Data/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/airport-codes-na.txt


In [20]:
val airports = spark.read
.option("header", "true")
.option("inferSchema","true")
.option("delimiter", "\t")
.csv(airportsPath)

airports.createOrReplaceTempView("airports_na")

airports: org.apache.spark.sql.DataFrame = [City: string, State: string ... 2 more fields]


In [21]:
val delays = spark.read
.option("header","true")
.csv(delaysPath)
.withColumn("delay", expr("CAST(delay as INT) as delay"))
.withColumn("distance", expr("CAST(distance as INT) as distance"))

delays.createOrReplaceTempView("departureDelays")

delays: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 3 more fields]


In [27]:
val foo = delays.filter(expr("""origin == 'SEA' AND destination == 'SFO' AND date like '01010%' AND delay > 0"""))
foo.createOrReplaceTempView("foo")

foo: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: string, delay: int ... 3 more fields]


In [6]:
spark.sql("SELECT * FROM airports_na LIMIT 10").show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+



In [7]:
spark.sql("SELECT * FROM departureDelays LIMIT 10").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+



In [8]:
spark.sql("SELECT * FROM foo").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [9]:
val bar = delays.union(foo)
bar.createOrReplaceTempView("bar")
bar.filter(expr("""origin == 'SEA' AND destination == 'SFO' AND date LIKE '01010%' AND delay > 0""")).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



bar: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: string, delay: int ... 3 more fields]


In [10]:
spark.sql("""
SELECT * 
 FROM bar 
 WHERE origin = 'SEA' 
 AND destination = 'SFO' 
 AND date LIKE '01010%' 
 AND delay > 0
""").show()


+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [11]:
foo.join(airports.as("air"), $"air.IATA" === $"origin").select("City", "State", "date", "delay", "distance", "destination").show()

+-------+-----+--------+-----+--------+-----------+
|   City|State|    date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
|Seattle|   WA|01010710|   31|     590|        SFO|
|Seattle|   WA|01010955|  104|     590|        SFO|
|Seattle|   WA|01010730|    5|     590|        SFO|
+-------+-----+--------+-----+--------+-----------+



In [12]:
spark.sql("""
SELECT a.City, a.State, f.date, f.delay, f.distance, f.destination 
 FROM foo f
 JOIN airports_na a
 ON a.IATA = f.origin
""").show()

+-------+-----+--------+-----+--------+-----------+
|   City|State|    date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
|Seattle|   WA|01010710|   31|     590|        SFO|
|Seattle|   WA|01010955|  104|     590|        SFO|
|Seattle|   WA|01010730|    5|     590|        SFO|
+-------+-----+--------+-----+--------+-----------+



In [16]:
spark.stop()

In [17]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
val spark = SparkSession.builder.appName("SparkSQLExampleApp").enableHiveSupport().getOrCreate()

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@339a6673


In [14]:
spark.sql("DROP TABLE IF EXISTS departureDelaysWindow")

res12: org.apache.spark.sql.DataFrame = []


In [22]:
spark.sql("""CREATE TABLE departureDelaysWindow AS
SELECT origin, destination, SUM(delay) AS TotalDelays
 FROM departureDelays
WHERE origin IN ('SEA', 'SFO', 'JFK')
 AND destination IN ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL')
GROUP BY origin, destination""")

res18: org.apache.spark.sql.DataFrame = [origin: string, destination: string ... 1 more field]


In [23]:
spark.sql("""SELECT * FROM departureDelaysWindow""").show()

+------+-----------+-----------+
|origin|destination|TotalDelays|
+------+-----------+-----------+
|   JFK|        ORD|       5608|
|   SEA|        LAX|       9359|
|   JFK|        SFO|      35619|
|   SFO|        ORD|      27412|
|   JFK|        DEN|       4315|
|   SFO|        DEN|      18688|
|   SFO|        SEA|      17080|
|   SEA|        SFO|      22293|
|   JFK|        ATL|      12141|
|   SFO|        ATL|       5091|
|   SEA|        DEN|      13645|
|   SEA|        ATL|       4535|
|   SEA|        ORD|      10041|
|   JFK|        SEA|       7856|
|   JFK|        LAX|      35755|
|   SFO|        JFK|      24100|
|   SFO|        LAX|      40798|
|   SEA|        JFK|       4667|
+------+-----------+-----------+



In [25]:
spark.sql("""SELECT origin, destination, sum(TotalDelays) as TotalDelays
             FROM departureDelaysWindow
             WHERE origin = 'SEA'
             GROUP BY origin, destination
             ORDER BY sum(TotalDelays) desc
             LIMIT 3""").show()

+------+-----------+-----------+
|origin|destination|TotalDelays|
+------+-----------+-----------+
|   SEA|        SFO|      22293|
|   SEA|        DEN|      13645|
|   SEA|        ORD|      10041|
+------+-----------+-----------+



In [26]:
spark.sql("""SELECT origin, destination, TotalDelays, rank
             FROM (
                 SELECT origin, destination, TotalDelays, dense_rank()
                         OVER(PARTITION BY origin ORDER BY TotalDelays DESC) as rank
                         FROM departureDelaysWindow)
             WHERE rank <= 3""").show()

+------+-----------+-----------+----+
|origin|destination|TotalDelays|rank|
+------+-----------+-----------+----+
|   SEA|        SFO|      22293|   1|
|   SEA|        DEN|      13645|   2|
|   SEA|        ORD|      10041|   3|
|   SFO|        LAX|      40798|   1|
|   SFO|        ORD|      27412|   2|
|   SFO|        JFK|      24100|   3|
|   JFK|        LAX|      35755|   1|
|   JFK|        SFO|      35619|   2|
|   JFK|        ATL|      12141|   3|
+------+-----------+-----------+----+



In [28]:
foo.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [29]:
import org.apache.spark.sql.functions.expr
val foo2 = foo.withColumn("status", expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END"))

import org.apache.spark.sql.functions.expr
foo2: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 4 more fields]


In [30]:
foo2.show()

+--------+-----+--------+------+-----------+-------+
|    date|delay|distance|origin|destination| status|
+--------+-----+--------+------+-----------+-------+
|01010710|   31|     590|   SEA|        SFO|Delayed|
|01010955|  104|     590|   SEA|        SFO|Delayed|
|01010730|    5|     590|   SEA|        SFO|On-time|
+--------+-----+--------+------+-----------+-------+



In [31]:
val foo3 = foo2.drop("delay")
foo3.show()

+--------+--------+------+-----------+-------+
|    date|distance|origin|destination| status|
+--------+--------+------+-----------+-------+
|01010710|     590|   SEA|        SFO|Delayed|
|01010955|     590|   SEA|        SFO|Delayed|
|01010730|     590|   SEA|        SFO|On-time|
+--------+--------+------+-----------+-------+



foo3: org.apache.spark.sql.DataFrame = [date: string, distance: int ... 3 more fields]


In [32]:
val foo4 = foo3.withColumnRenamed("status", "flight_status")
foo4.show()

+--------+--------+------+-----------+-------------+
|    date|distance|origin|destination|flight_status|
+--------+--------+------+-----------+-------------+
|01010710|     590|   SEA|        SFO|      Delayed|
|01010955|     590|   SEA|        SFO|      Delayed|
|01010730|     590|   SEA|        SFO|      On-time|
+--------+--------+------+-----------+-------------+



foo4: org.apache.spark.sql.DataFrame = [date: string, distance: int ... 3 more fields]


In [35]:
spark.sql("""SELECT destination, cast(substring(date, 0, 2) as int) as month, delay
            FROM departureDelays
            WHERE origin = 'SEA'""").show()

+-----------+-----+-----+
|destination|month|delay|
+-----------+-----+-----+
|        ORD|    1|   92|
|        JFK|    1|   -7|
|        DFW|    1|   -5|
|        MIA|    1|   -3|
|        DFW|    1|   -3|
|        DFW|    1|    1|
|        ORD|    1|  -10|
|        DFW|    1|   -6|
|        DFW|    1|   -2|
|        ORD|    1|   -3|
|        ORD|    1|    0|
|        DFW|    1|   23|
|        DFW|    1|   36|
|        ORD|    1|  298|
|        JFK|    1|    4|
|        DFW|    1|    0|
|        MIA|    1|    2|
|        DFW|    1|    0|
|        DFW|    1|    0|
|        ORD|    1|   83|
+-----------+-----+-----+
only showing top 20 rows



In [37]:
spark.sql("""SELECT * FROM(
                SELECT destination, cast(substring(date,0,2)as int) as month, delay
                FROM departureDelays
                Where origin = 'SEA')
                PIVOT(
                    cast(avg(delay) as decimal(4,2)) as AvgDelay, max(delay) as MaxDelay FOR month in (1 Jan, 2 Feb)
                    )
                ORDER BY destination""").show()

+-----------+------------+------------+------------+------------+
|destination|Jan_AvgDelay|Jan_MaxDelay|Feb_AvgDelay|Feb_MaxDelay|
+-----------+------------+------------+------------+------------+
|        ABQ|       19.86|         316|       11.42|          69|
|        ANC|        4.44|         149|        7.90|         141|
|        ATL|       11.98|         397|        7.73|         145|
|        AUS|        3.48|          50|       -0.21|          18|
|        BOS|        7.84|         110|       14.58|         152|
|        BUR|       -2.03|          56|       -1.89|          78|
|        CLE|       16.00|          27|        null|        null|
|        CLT|        2.53|          41|       12.96|         228|
|        COS|        5.32|          82|       12.18|         203|
|        CVG|       -0.50|           4|        null|        null|
|        DCA|       -1.15|          50|        0.07|          34|
|        DEN|       13.13|         425|       12.95|         625|
|        D

error: error while loading Decimal, class file 'C:\Spark\jars\spark-catalyst_2.12-3.0.3.jar(org/apache/spark/sql/types/Decimal.class)' is broken
(class java.lang.RuntimeException/error reading Scala signature of Decimal.class: assertion failed:
  Decimal$DecimalIsFractional
     while compiling: <console>
        during phase: globalPhase=terminal, enteringPhase=jvm
     library version: version 2.12.10
    compiler version: version 2.12.10
  reconstructed args: -classpath  -Yrepl-class-based -Yrepl-outdir C:\Users\ALVARO~1.ROM\AppData\Local\Temp\tmps14mg5xl

  last tree to typer: TypeTree(class Byte)
       tree position: line 6 of <console>
            tree tpe: Byte
              symbol: (final abstract) class Byte in package scala
   symbol definition: final abstract class Byte extends  (a ClassSymbol)
      symbol package: scala
       symbol owners: class Byte
           call site: constructor $eval in object $eval in package $line50

== Source file context for tree position ==



In [ ]:

/*
    Pros y contras de utilizar UDFs:
    -Pros:
        -Pueden realizar operaciones de forma más rápida que las funciones predefinidas.
        -Se puden usar directamente en Spark SQL.
        -Operan por sesión, por lo que no persistirán en el metastore.
    
    -Cons:
        -Spark SQl no garantiza el orden de evaluación de subexpressiones, por lo que para hacer el chequeo de nulos
         hay que hacer el chequeo de nulos dentro del UDF o usar un if o case when.
        
*/